In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPool1D
from tensorflow.keras.optimizers import Adam
print(tf.__version__)

2.5.0


In [ ]:
import pandas as pd
import numpy as np
import  seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/능지

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[Errno 2] No such file or directory: 'gdrive/MyDrive/능지'
/content/gdrive/MyDrive/능지


In [ ]:

data = pd.read_csv('creditcard.csv')
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
data.shape

(284807, 31)

In [ ]:
data['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [ ]:
non_fraud = data[data['Class']==0]
fraud = data[data['Class']==1]

In [ ]:

non_fraud.shape, fraud.shape

((284315, 31), (492, 31))

In [ ]:
non_fraud = non_fraud.sample(fraud.shape[0])
non_fraud.shape

(492, 31)

In [ ]:

data = fraud.append(non_fraud, ignore_index=True)
data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,406.0,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,-2.772272,3.202033,-2.899907,-0.595222,-4.289254,0.389724,-1.140747,-2.830056,-0.016822,0.416956,0.126911,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,0.00,1
1,472.0,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,-0.838587,-0.414575,-0.503141,0.676502,-1.692029,2.000635,0.666780,0.599717,1.725321,0.283345,2.102339,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.00,1
2,4462.0,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,-1.525412,2.032912,-6.560124,0.022937,-1.470102,-0.698826,-2.282194,-4.781831,-2.615665,-1.334441,-0.430022,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,239.93,1
3,6986.0,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,-4.801637,4.895844,-10.912819,0.184372,-6.771097,-0.007326,-7.358083,-12.598419,-5.131549,0.308334,-0.171608,0.573574,0.176968,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,59.00,1
4,7519.0,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,-2.447469,2.101344,-4.609628,1.464378,-6.079337,-0.339237,2.581851,6.739384,3.042493,-2.721853,0.009061,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,1.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,135147.0,-0.619994,0.972530,1.910218,2.316297,0.432579,0.888768,0.580786,0.359177,-1.301540,0.532313,-0.330692,-0.165833,-0.418845,-0.143777,-1.992465,1.687555,-1.564158,0.795084,-2.004199,-0.152585,0.323177,0.791883,-0.114625,-0.379786,-0.354698,-0.116223,0.134285,0.171422,59.93,0
980,135076.0,-0.772375,1.685787,-1.129218,-0.512996,0.294611,-0.786494,0.319838,0.643834,-0.289474,-0.456610,-1.150604,0.649145,0.973042,0.807713,0.403576,-0.254176,-0.205795,0.108329,-0.017996,-0.038437,0.423860,1.307688,-0.002292,0.658511,-0.643767,-0.236478,0.369650,0.263504,0.79,0
981,168678.0,-0.508689,0.356332,-0.518106,-0.427701,0.532718,-0.801127,0.701437,0.242166,-0.285375,-0.866798,-1.248447,0.349337,0.223760,0.428628,-0.715704,-0.078010,-0.206974,-0.644935,-0.007654,0.009297,-0.024553,-0.388234,0.335984,0.741116,-0.691876,0.107269,-0.087355,-0.019989,78.77,0
982,40418.0,-3.524526,2.558364,1.350300,0.040051,-1.334421,0.021723,-0.558273,0.112471,1.334919,2.295002,1.458439,1.235840,0.995690,-0.806488,0.689881,0.564001,-0.974273,0.939828,-0.454058,0.045589,0.569333,1.159584,-0.051793,0.537987,-0.154503,-0.573721,-2.346308,-0.262116,3.79,0


In [ ]:

data['Class'].value_counts()

1    492
0    492
Name: Class, dtype: int64

In [ ]:

X = data.drop('Class', axis = 1)
y = data['Class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)


In [ ]:
X_train.shape, X_test.shape


((787, 30), (197, 30))

In [ ]:

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
epochs = 20
model = Sequential()
model.add(Conv1D(32, 2, activation='relu', input_shape = X_train[0].shape))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv1D(64, 2, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

In [ ]:

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 29, 32)            96        
_________________________________________________________________
batch_normalization_2 (Batch (None, 29, 32)            128       
_________________________________________________________________
dropout_3 (Dropout)          (None, 29, 32)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 28, 64)            4160      
_________________________________________________________________
batch_normalization_3 (Batch (None, 28, 64)            256       
_________________________________________________________________
dropout_4 (Dropout)          (None, 28, 64)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1792)             

In [ ]:
model.compile(optimizer=Adam(lr=0.0001), loss = 'binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
history = model.fit(X_train, y_train,batch_size = 1, epochs=epochs, validation_data=(X_test, y_test), verbose=1)

Epoch 1/20
787/787 [==============================] - 4s 3ms/step - loss: 0.4662 - accuracy: 0.8272 - val_loss: 0.2662 - val_accuracy: 0.8985
Epoch 2/20
787/787 [==============================] - 2s 3ms/step - loss: 0.3110 - accuracy: 0.8793 - val_loss: 0.2646 - val_accuracy: 0.8934
Epoch 3/20
787/787 [==============================] - 2s 3ms/step - loss: 0.2544 - accuracy: 0.9174 - val_loss: 0.2381 - val_accuracy: 0.9137
Epoch 4/20
787/787 [==============================] - 2s 3ms/step - loss: 0.2531 - accuracy: 0.9111 - val_loss: 0.2483 - val_accuracy: 0.9036
Epoch 5/20
787/787 [==============================] - 2s 3ms/step - loss: 0.1947 - accuracy: 0.9314 - val_loss: 0.2995 - val_accuracy: 0.9188
Epoch 6/20
787/787 [==============================] - 2s 3ms/step - loss: 0.1788 - accuracy: 0.9352 - val_loss: 0.2972 - val_accuracy: 0.9086
Epoch 7/20
787/787 [==============================] - 3s 3ms/step - loss: 0.1865 - accuracy: 0.9339 - val_loss: 0.2914 - val_accuracy: 0.9086
Epoch 